In [0]:
import pandas as pd

# Hate-speech, offensive language, neither

In [0]:
davidson = pd.read_csv("labeled_data.csv")

In [0]:
davidson.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [0]:
davidson.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,24783.0,12681.192027,7299.553863,0.0,6372.5,12703.0,18995.5,25296.0
count,24783.0,3.243473,0.883060,3.0,3.0,3.0,3.0,9.0
hate_speech,24783.0,0.280515,0.631851,0.0,0.0,0.0,0.0,7.0
offensive_language,24783.0,2.413711,1.399459,0.0,2.0,3.0,3.0,9.0
neither,24783.0,0.549247,1.113299,0.0,0.0,0.0,0.0,9.0
class,24783.0,1.110277,0.462089,0.0,1.0,1.0,1.0,2.0


#### Data guide:

- count = number of CrowdFlower users who coded each tweet (min is 3, sometimes more users coded a tweet when judgments were determined to be unreliable by CF).

- hate_speech = number of CF users who judged the tweet to be hate speech.

- offensive_language = number of CF users who judged the tweet to be offensive.

- neither = number of CF users who judged the tweet to be neither offensive nor non-offensive.

- class = class label for majority of CF users.
    - 0 - hate speech,
    - 1 - offensive language,
    - 2 - neither

# Sexism, racism, both, neither

In [0]:
wassem_css = pd.read_csv("wassem_css.csv", sep="\t", index_col=0, dtype={'Tweet_ID': int,
                                                                         'Previous': str,
                                                                         'User_ID': str,
                                                                         'Text': str,
                                                                         'Expert_Annotation': str, 
                                                                        'Amateur_Annotations': str})
wassem_css.head()

,Previous,User_ID,Text,Expert_Annotation,Amateur_Annotations
Tweet_ID,,,,,
570775709343461377,570775111814373376,24414732,@TVBachelor @NewJerzeyBoy what about the femin...,both,"[neither,sexism,sexism]"
596730741490253824,596729971009196032,13857342,@ThelmaSleaze uh... did you watch the video? o...,both,"[neither,racism,sexism]"
600800567426883585,NaN,254174483,Managed to hit a bird and a small rodent on my...,both,"[neither,neither,neither]"
603546751841230848,NaN,2264363900,"If you believe a #holohoax or #gays, LGBT &amp...",both,"[sexism,both,sexism]"
603584564582326272,NaN,2198745044,"Rid yourself of #Feminazi, #Fag &amp; #Jewf. E...",both,"[sexism,sexism,sexism]"


In [0]:
# pb: Amateur_Annotations is a string
wassem_css['Amateur_Annotations'] = wassem_css['Amateur_Annotations'].apply(lambda x:
                                                                            x.split('[')[1].split(']')[0].split(","))

In [0]:
target = pd.DataFrame(wassem_css['Amateur_Annotations'].values.tolist(),
                      index=wassem_css['Amateur_Annotations'].index).dropna(axis=1).add_prefix("user_")
target = pd.concat([pd.get_dummies(target.user_0),
                    pd.get_dummies(target.user_1),
                    pd.get_dummies(target.user_2)], axis=0).groupby('Tweet_ID').sum()
del target["link"]
target['Amateur_Annotation'] = target.idxmax(axis=1)
target.head()

,both,neither,racism,sexism,Amateur_Annotation
Tweet_ID,,,,,
319399851215433729,0,1,1,1,neither
320817818222358529,0,3,0,0,neither
324114200450437120,0,1,2,0,racism
326286656854454273,0,3,0,0,neither
437874382954704896,0,3,0,0,neither


In [0]:
wassem_css = pd.concat([wassem_css, target], join='inner', axis=1)
wassem_css.head()

,Previous,User_ID,Text,Expert_Annotation,Amateur_Annotations,both,neither,racism,sexism,Amateur_Annotation
Tweet_ID,,,,,,,,,,
570775709343461377,570775111814373376,24414732,@TVBachelor @NewJerzeyBoy what about the femin...,both,"[neither, sexism, sexism]",0,1,0,2,sexism
596730741490253824,596729971009196032,13857342,@ThelmaSleaze uh... did you watch the video? o...,both,"[neither, racism, sexism]",0,1,1,1,neither
600800567426883585,NaN,254174483,Managed to hit a bird and a small rodent on my...,both,"[neither, neither, neither]",0,3,0,0,neither
603546751841230848,NaN,2264363900,"If you believe a #holohoax or #gays, LGBT &amp...",both,"[sexism, both, sexism]",1,0,0,2,sexism
603584564582326272,NaN,2198745044,"Rid yourself of #Feminazi, #Fag &amp; #Jewf. E...",both,"[sexism, sexism, sexism]",0,0,0,3,sexism


In [0]:
pd.crosstab(wassem_css.Expert_Annotation, wassem_css.Amateur_Annotation, margins=True)

Amateur_Annotation,both,neither,racism,sexism,All
Expert_Annotation,,,,,
both,9,11,3,12,35
neither,45,5146,81,457,5729
racism,2,50,34,2,88
sexism,9,161,6,639,815
All,65,5368,124,1110,6667


In [0]:
# interesting to see the difference of point of view between amateur and expert
# almost half of the sexist tweets from amateur are classified as non-sexist for the expert (457/1110)
# more than the half of the racist tweets from expert are classified as non-racist for the amateurs (50/88)

# Sexism, racism, none

In [0]:
wassem_hovy_naacl = pd.read_csv("wassem_hovy_naacl.csv", sep="\t", index_col=0)
wassem_hovy_naacl.head()

,Previous,User_ID,Text,Label
Tweet_ID,,,,
567080871472017408,5.670786e+17,2941145694,@truaemusic The followers of the religion give...,racism
566098518616449024,5.660874e+17,2941145694,@Love___Egypt @washingtonpost All the land tha...,racism
554672350361571328,NaN,2941145694,@CamilleHughes16 @edbarber101658 @_Watch_Obser...,racism
569878129772081153,5.698451e+17,2941145694,@KellyTurner99 @brushyblues @buellerishere The...,racism
568541747849998337,NaN,2941145694,@buellerishere Who cares. ISIS does what Moham...,racism


In [0]:
wassem_hovy_naacl.Label.value_counts()

none      11115
sexism     3148
racism     1939
Name: Label, dtype: int64

# Homophobia

In [0]:
homophobia = pd.read_csv("homophobes_tweets_checked.csv")
homophobia.head()

,created_at,favorites,retweets,text,tweet_id,user_id,user_name,user_screen_name
0,1.438032e+12,55,22,American Voices: Boy Scouts To Lift Ban On Gay...,"6,25778285025346E+017",318831260,Homophobes,318831260
1,1.421182e+12,169,119,Homosexuality is a disease!pic.twitter.com/Re2...,"5,55102898556637E+017",318831260,Homophobes,318831260
2,1.416594e+12,12,12,Gay Man Unaware He Focus Of Thousands Of Praye...,"5,35859739163243E+017",318831260,Homophobes,318831260
3,1.395821e+12,5,4,"Stop smearing feces on the wall, homosexuals! ...","4,48730038905479E+017",318831260,Homophobes,318831260
4,1.374696e+12,5,16,A Straight Man Asks How to Ditch a Gay Friend ...,"3,6012626215451E+017",318831260,Homophobes,318831260


In [0]:
homophobia['label'] = 'homophobia'

In [0]:
homophobia.head()

,created_at,favorites,retweets,text,tweet_id,user_id,user_name,user_screen_name,label
0,1.438032e+12,55,22,American Voices: Boy Scouts To Lift Ban On Gay...,"6,25778285025346E+017",318831260,Homophobes,318831260,homophobia
1,1.421182e+12,169,119,Homosexuality is a disease!pic.twitter.com/Re2...,"5,55102898556637E+017",318831260,Homophobes,318831260,homophobia
2,1.416594e+12,12,12,Gay Man Unaware He Focus Of Thousands Of Praye...,"5,35859739163243E+017",318831260,Homophobes,318831260,homophobia
3,1.395821e+12,5,4,"Stop smearing feces on the wall, homosexuals! ...","4,48730038905479E+017",318831260,Homophobes,318831260,homophobia
4,1.374696e+12,5,16,A Straight Man Asks How to Ditch a Gay Friend ...,"3,6012626215451E+017",318831260,Homophobes,318831260,homophobia


# Merge in one dataset

In [0]:
# wassem css: keep sexism and neither
wassem_css.Expert_Annotation.unique()

array(['both', 'racism', 'sexism', 'neither'], dtype=object)

In [0]:
wassem_css.reset_index(inplace=True)

In [0]:
sexism = wassem_css.loc[wassem_css.Expert_Annotation.isin(['sexism', 'neither']),
                        ["Tweet_ID", "Text", "Expert_Annotation"]]

In [0]:
# wassem_hovy_naacl: keep sexism and none
wassem_hovy_naacl.Label.unique()

array(['racism', 'sexism', 'none'], dtype=object)

In [0]:
wassem_hovy_naacl.reset_index(inplace=True)

In [0]:
sexism2 = wassem_hovy_naacl.loc[wassem_hovy_naacl.Label.isin(['sexism', 'none']),
                      ["Tweet_ID", "Text", "Label"]]

In [0]:
# homophobia: keep all rows but only the columns tweet_id, text, label
homophobia = homophobia.loc[:, ["tweet_id", "text", "label"]]

### Rename columns and merge

In [0]:
sexism.rename(columns={'Tweet_ID': 'tweet_id', 'Text': 'text', 'Expert_Annotation': 'label'}, inplace=True)
sexism['label'] = sexism.label.str.replace('neither', 'none')
sexism2.rename(columns={'Tweet_ID': 'tweet_id', 'Text': 'text', 'Label': 'label'}, inplace=True)

In [0]:
display(sexism.dtypes)
display(sexism2.dtypes)
display(homophobia.dtypes)

tweet_id     int64
text        object
label       object
dtype: object

tweet_id     int64
text        object
label       object
dtype: object

tweet_id    object
text        object
label       object
dtype: object

In [0]:
sexism = sexism.astype(str)
sexism2 = sexism2.astype(str)
homophobia = homophobia.astype(str)

In [0]:
display(sexism.dtypes)
display(sexism2.dtypes)
display(homophobia.dtypes)

tweet_id    object
text        object
label       object
dtype: object

tweet_id    object
text        object
label       object
dtype: object

tweet_id    object
text        object
label       object
dtype: object

In [0]:
data = pd.concat([sexism, sexism2, homophobia], axis=0)

In [0]:
data.head()

,tweet_id,text,label
123,576424537065156608,@AAmbrosim a ta. Kkkk kkkk as feminazi,sexism
124,570757526846881792,@Javanzord @Douglas_Feer feminazi,sexism
125,576536812283269121,bro my icon is literally feminazi misanandrist...,sexism
126,576428713023143936,"RT @PhilGonc: @Independent ""Stop Male Violence...",sexism
127,572336406053322753,"Sauce, capsicum and stock from a packet / jar....",sexism


In [0]:
data.reset_index(drop=True, inplace=True)

In [0]:
data.label.value_counts()

none          16844
sexism         3963
homophobia       87
Name: label, dtype: int64

In [0]:
data.head()

,tweet_id,text,label
0,576424537065156608,@AAmbrosim a ta. Kkkk kkkk as feminazi,sexism
1,570757526846881792,@Javanzord @Douglas_Feer feminazi,sexism
2,576536812283269121,bro my icon is literally feminazi misanandrist...,sexism
3,576428713023143936,"RT @PhilGonc: @Independent ""Stop Male Violence...",sexism
4,572336406053322753,"Sauce, capsicum and stock from a packet / jar....",sexism


In [0]:
data.tail()

,tweet_id,text,label
20889,"1,86961697997722E+017",In case you missed it: A Homophobe's Guide for...,homophobia
20890,"1,86629008824349E+017",A Homophobe's Guide for Dealing with a Gay Chi...,homophobia
20891,"1,8563977082583E+017",Joey Mitchell (@basementatheist) reads through...,homophobia
20892,"1,82386216543601E+017",Why I Hate Gay People. http://www.youtube.com...,homophobia
20893,"1,7840229184453E+017",Stop making up your own rules! Do things the w...,homophobia


In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20894 entries, 0 to 20893
Data columns (total 3 columns):
tweet_id    20894 non-null object
text        20894 non-null object
label       20894 non-null object
dtypes: object(3)
memory usage: 489.8+ KB


In [0]:
data.to_csv('detect_hate_speech_data.csv', sep='|')